In [1]:
from lsst.daf.butler import Butler

butler = Butler.from_config(
    "/repo/dp1",
    instrument="LSSTComCam",
    skymap="lsst_cells_v1",
    collections="LSSTComCam/DP1",
    writeable=True,
)

In [2]:
camera = butler.get("camera", instrument="LSSTComCam")

In [3]:
from importlib import reload

import lsst.obs.base.visit_geometry

reload(lsst.obs.base.visit_geometry)
VisitGeometry = lsst.obs.base.visit_geometry.VisitGeometry

In [4]:
with butler.query() as query:
    query = query.join_dataset_search("preliminary_visit_summary")
    visits = VisitGeometry.from_query(query)

In [5]:
from tqdm.notebook import tqdm

In [6]:
n = -1

In [9]:
from matplotlib import pyplot

figure = pyplot.figure(figsize=(10, 10))


def fix_and_plot_visit(visit):
    figure.clear()
    axes, proj = visit.plot(
        figure=figure,
        fit_region_style=dict(fill=True, facecolor="b", edgecolor="b", alpha=0.2),
        raw_region_style=dict(fill=False, edgecolor="r"),
        fallback_region_style=dict(fill=False, edgecolor="g"),
        raw_axes_style=dict(color="r", linestyle=":"),
        fallback_axes_style=dict(color="g", linestyle=":"),
        detector_record_style=dict(fill=False, edgecolor="m", linestyle=":"),
        visit_record_style=dict(fill=False, edgecolor="m"),
        plot_all_fallback=True,
    )
    visit.fix_records()
    visit.plot(
        axes=axes,
        proj=proj,
        detector_record_style=dict(fill=False, edgecolor="c", linestyle=":"),
        visit_record_style=dict(fill=False, edgecolor="c"),
    )


for n, visit in enumerate(tqdm(visits[n + 1 :], initial=n + 1, total=len(visits)), start=n + 1):
    visit.load_detector_geometry(butler, camera=camera)
    if visit.get_missing_detectors():
        fix_and_plot_visit(visit)
        break
    else:
        visit.fix_records()


 43%|####3     | 778/1792 [00:00<?, ?it/s]

<Figure size 1000x1000 with 0 Axes>

In [10]:
VisitGeometry.replace_butler_records(butler, visits)